<a href="https://colab.research.google.com/github/Alexandremsn/demand/blob/main/cff_model__P6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
pd.set_option('display.max_columns', 500)
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import numpy as np

importando as bibliotecas

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


acessando o google drive
importando o aqruivo e testando sua integridade

In [3]:
testdf = pd.read_csv('/content/drive/My Drive/cffiles/testdf_trans.csv')
testdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17280 entries, 0 to 17279
Data columns (total 43 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   17280 non-null  int64  
 1   date                 17280 non-null  object 
 2   store_nbr            17280 non-null  int64  
 3   item_nbr             17280 non-null  int64  
 4   onpromotion          17280 non-null  int64  
 5   oil_price            17280 non-null  float64
 6   perishable           17280 non-null  int64  
 7   Friday               17280 non-null  float64
 8   Monday               17280 non-null  float64
 9   Saturday             17280 non-null  float64
 10  Sunday               17280 non-null  float64
 11  Thursday             17280 non-null  float64
 12  Tuesday              17280 non-null  float64
 13  Wednesday            17280 non-null  float64
 14  is_additional        17280 non-null  float64
 15  is_black_friday      17280 non-null 

importando o arquivo de treino e testando sua integridade

In [4]:
traindf = pd.read_csv('/content/drive/My Drive/cffiles/traindf_trans_new.csv')
traindf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1634358 entries, 0 to 1634357
Data columns (total 44 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   id                   1634358 non-null  int64  
 1   date                 1634358 non-null  object 
 2   store_nbr            1634358 non-null  int64  
 3   item_nbr             1634358 non-null  int64  
 4   unit_sales           1634358 non-null  int64  
 5   onpromotion          1634358 non-null  int64  
 6   oil_price            1634358 non-null  float64
 7   perishable           1634358 non-null  int64  
 8   Friday               1634358 non-null  float64
 9   Monday               1634358 non-null  float64
 10  Saturday             1634358 non-null  float64
 11  Sunday               1634358 non-null  float64
 12  Thursday             1634358 non-null  float64
 13  Tuesday              1634358 non-null  float64
 14  Wednesday            1634358 non-null  float64
 15

transformando o formato de data

In [5]:
traindf.date = pd.to_datetime(traindf.date, format='%Y-%m-%d')
testdf.date = pd.to_datetime(testdf.date, format='%Y-%m-%d')

In [6]:
testdf.describe()

,id,store_nbr,item_nbr,onpromotion,oil_price,perishable,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,is_additional,is_black_friday,is_cyber_monday,is_holiday,is_holiday_eve,is_natural_disaster,is_world_cup,Madre,Navidad,BEVERAGES,BREAD/BAKERY,CLEANING,GROCERY I,PERSONAL CARE,1004,1010,1012,1014,1040,1058,1072,1084,1120,1124,2702,3032,3038,4114,transactions
count,1.728000e+04,17280.000000,1.728000e+04,17280.000000,1.728000e+04,17280.000000,17280.000000,17280.000000,17280.000000,17280.000000,17280.000000,17280.000000,17280.000000,17280.0,17280.0,17280.0,17280.000000,17280.0,17280.0,17280.0,17280.0,17280.0,17280.000000,17280.000000,17280.000000,17280.000000,17280.000000,17280.000000,17280.000000,17280.000000,17280.000000,17280.000000,17280.000000,17280.000000,17280.000000,17280.000000,17280.000000,17280.000000,17280.000000,17280.000000,17280.000000,17280.000000
mean,1.271810e+08,27.500000,6.041492e+05,0.094155,4.815138e+01,0.050000,0.125000,0.125000,0.125000,0.125000,0.187500,0.125000,0.187500,0.0,0.0,0.0,0.005787,0.0,0.0,0.0,0.0,0.0,0.100000,0.050000,0.100000,0.700000,0.050000,0.050000,0.050000,0.050000,0.100000,0.100000,0.050000,0.250000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,26316.448071
std,9.729967e+05,15.586235,2.792820e+05,0.292053,1.421127e-14,0.217951,0.330728,0.330728,0.330728,0.330728,0.390324,0.330728,0.390324,0.0,0.0,0.0,0.075854,0.0,0.0,0.0,0.0,0.0,0.300009,0.217951,0.300009,0.458271,0.217951,0.217951,0.217951,0.217951,0.300009,0.300009,0.217951,0.433025,0.217951,0.217951,0.217951,0.217951,0.217951,0.217951,0.217951,15147.473420
min,1.254972e+08,1.000000,2.583960e+05,0.000000,4.815138e+01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.090798
25%,1.263392e+08,14.000000,3.520505e+05,0.000000,4.815138e+01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,16936.668945
50%,1.271812e+08,27.500000,5.622015e+05,0.000000,4.815138e+01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,21070.560547
75%,1.280231e+08,41.000000,8.433472e+05,0.000000,4.815138e+01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,32104.894531
max,1.288651e+08,54.000000,1.084881e+06,1.000000,4.815138e+01,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,0.0,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,72882.742188


In [7]:
traindf.item_nbr.value_counts()

502331     83475
314384     83450
364606     83308
265559     83047
559870     82513
1036689    82134
273528     82108
564533     82086
261052     81774
414353     81755
1084881    81324
839363     81268
850333     81097
414750     81091
847863     80947
581078     80776
1047679    80713
258396     80574
759893     80499
841842     80419
Name: item_nbr, dtype: int64

In [8]:
dummiesdf = pd.get_dummies(traindf['store_nbr'].astype(str))
dummiestest = pd.get_dummies(testdf['store_nbr'].astype(str))


criando variáveis dummies para as lojas para que nosso modelo entenda a diferença entre lojas

In [9]:
traindf_dummies = pd.concat([traindf,dummiesdf],axis=1)
testdf_dummies = pd.concat([testdf,dummiestest],axis=1)

In [10]:
traindf_dummies.columns

Index(['id', 'date', 'store_nbr', 'item_nbr', 'unit_sales', 'onpromotion',
       'oil_price', 'perishable', 'Friday', 'Monday', 'Saturday', 'Sunday',
       'Thursday', 'Tuesday', 'Wednesday', 'is_additional', 'is_black_friday',
       'is_cyber_monday', 'is_holiday', 'is_holiday_eve',
       'is_natural_disaster', 'is_world_cup', 'Madre', 'Navidad', 'BEVERAGES',
       'BREAD/BAKERY', 'CLEANING', 'GROCERY I', 'PERSONAL CARE', '1004',
       '1010', '1012', '1014', '1040', '1058', '1072', '1084', '1120', '1124',
       '2702', '3032', '3038', '4114', 'transactions', '1', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23',
       '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34',
       '35', '36', '37', '38', '39', '4', '40', '41', '42', '43', '44', '45',
       '46', '47', '48', '49', '5', '50', '51', '52', '53', '54', '6', '7',
       '8', '9'],
      dtype='object')

agora temos todas as variáveis que serão utilizadas em nosso modelo inicial

In [11]:
traindf_dummies['dif_days'] = traindf_dummies['date'].apply(lambda x: (x - traindf_dummies.date.iloc[0]) / np.timedelta64(1, 'D') )
testdf_dummies['dif_days'] = testdf_dummies['date'].apply(lambda x: (x - traindf_dummies.date.iloc[0]) / np.timedelta64(1, 'D') )


criamos uma variável cumulativa para que nosso modelo entenda a passagem do´tempo assim somando os dias de forma cumulativa do dia zero até o dia final

In [12]:
(traindf.groupby('item_nbr').max() - traindf.groupby('item_nbr').min()).head(50)

,id,date,store_nbr,unit_sales,onpromotion,oil_price,perishable,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,is_additional,is_black_friday,is_cyber_monday,is_holiday,is_holiday_eve,is_natural_disaster,is_world_cup,Madre,Navidad,BEVERAGES,BREAD/BAKERY,CLEANING,GROCERY I,PERSONAL CARE,1004,1010,1012,1014,1040,1058,1072,1084,1120,1124,2702,3032,3038,4114,transactions
item_nbr,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
258396,125495716,1687 days,53,2739,1,84.43,0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8353.0
261052,125495717,1687 days,53,2424,1,84.43,0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8353.0
265559,125495718,1687 days,53,14025,1,84.43,0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8353.0
273528,125495721,1687 days,53,2933,1,84.43,0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8353.0
314384,125495719,1687 days,53,610,1,84.43,0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8353.0
364606,125495723,1687 days,53,4634,1,84.43,0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8353.0
414353,125495735,1687 days,53,3158,1,84.43,0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8353.0
414750,125495738,1687 days,53,1851,1,84.43,0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8353.0
502331,125495743,1687 days,53,832,1,84.43,0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8353.0


agora verificamos o range das varíaveis agrupadas por item

In [13]:
def time_series_train_test_split(df,test_size,predict_feature):
  test_split = int(int(df.shape[0])*test_size)
  df_train = df[test_split:]
  df_test = df[:test_split]
  X_train = df_train.drop(predict_feature, axis=1)
  X_test = df_test.drop(predict_feature, axis=1)
  y_train = df_train[predict_feature]
  y_test = df_test[predict_feature]
  return X_train, X_test , y_train, y_test

agora temos igual ao notebook anterior a função de geração do df de treino e teste

selecionamos um item e testamos a variação dos hiperparâmetros e variáveis e camadas, e chegamos nos valores abaixo.

In [14]:
datatestmodel = traindf_dummies[traindf_dummies['item_nbr']==265559]


selected_features = ['unit_sales', 'onpromotion',
       'oil_price', 'perishable', 'Friday', 'Monday', 'Saturday', 'Sunday',
       'Thursday', 'Tuesday', 'Wednesday', 'is_additional', 'is_black_friday',
       'is_cyber_monday', 'is_holiday', 'is_holiday_eve',
       'is_natural_disaster', 'is_world_cup', 'Madre', 'Navidad', 'BEVERAGES',
       'BREAD/BAKERY', 'CLEANING', 'GROCERY I', 'PERSONAL CARE', '1004',
       '1010', '1012', '1014', '1040', '1058', '1072', '1084', '1120', '1124',
       '2702', '3032', '3038', '4114', 'transactions', '1', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23',
       '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34',
       '35', '36', '37', '38', '39', '4', '40', '41', '42', '43', '44', '45',
       '46', '47', '48', '49', '5', '50', '51', '52', '53', '54', '6', '7',
       '8', '9','dif_days']       
features = datatestmodel[selected_features]
features.index = datatestmodel['date']
num_scaler = StandardScaler()
features_scaled = features
features_scaled['oil_price'] = num_scaler.fit_transform(features_scaled[['oil_price']])
features_scaled['transactions'] = num_scaler.fit_transform(features_scaled[['transactions']])
features_scaled['dif_days'] = num_scaler.fit_transform(features_scaled[['dif_days']])
model = Sequential()
model.add(Dense(30, input_dim=len(selected_features)-1, activation="relu"))
model.add(Dense(20, activation="relu"))
model.add(Dense(10, activation="relu"))
model.add(Dense(1))
model.compile(
    loss='mean_squared_error', 
    optimizer='adam', 
    metrics=['accuracy'])

<ipython-input-14-f981209a7926>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_scaled['oil_price'] = num_scaler.fit_transform(features_scaled[['oil_price']])
<ipython-input-14-f981209a7926>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_scaled['transactions'] = num_scaler.fit_transform(features_scaled[['transactions']])
<ipython-input-14-f981209a7926>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer

In [15]:
X_train, X_test , y_train, y_test = time_series_train_test_split(features_scaled,0.2,'unit_sales')

verificamos se não temos valores discrepantes nos dados que nosso modelo vai usar

In [16]:
datatestmodel.describe()

,id,store_nbr,item_nbr,unit_sales,onpromotion,oil_price,perishable,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,is_additional,is_black_friday,is_cyber_monday,is_holiday,is_holiday_eve,is_natural_disaster,is_world_cup,Madre,Navidad,BEVERAGES,BREAD/BAKERY,CLEANING,GROCERY I,PERSONAL CARE,1004,1010,1012,1014,1040,1058,1072,1084,1120,1124,2702,3032,3038,4114,transactions,1,10,11,12,13,14,15,16,17,18,19,2,20,21,22,23,24,25,26,27,28,29,3,30,31,32,33,34,35,36,37,38,39,4,40,41,42,43,44,45,46,47,48,49,5,50,51,52,53,54,6,7,8,9,dif_days
count,8.304700e+04,83047.000000,83047.0,83047.000000,83047.000000,83047.000000,83047.0,83047.000000,83047.000000,83047.000000,83047.000000,83047.000000,83047.000000,83047.000000,83047.000000,83047.000000,83047.000000,83047.000000,83047.000000,83047.000000,83047.000000,83047.000000,83047.000000,83047.0,83047.0,83047.0,83047.0,83047.0,83047.0,83047.0,83047.0,83047.0,83047.0,83047.0,83047.0,83047.0,83047.0,83047.0,83047.0,83047.0,83047.0,83047.0,83047.000000,83047.000000,83047.000000,83047.000000,83047.000000,83047.000000,83047.000000,83047.000000,83047.000000,83047.000000,83047.000000,83047.000000,83047.000000,83047.000000,83047.000000,83047.000000,83047.000000,83047.000000,83047.000000,83047.000000,83047.000000,83047.000000,83047.000000,83047.000000,83047.000000,83047.000000,83047.000000,83047.000000,83047.000000,83047.000000,83047.000000,83047.000000,83047.000000,83047.000000,83047.000000,83047.000000,83047.000000,83047.000000,83047.000000,83047.000000,83047.000000,83047.000000,83047.000000,83047.000000,83047.000000,83047.000000,83047.000000,83047.000000,83047.000000,83047.000000,83047.000000,83047.000000,83047.000000,83047.000000,83047.000000,83047.000000
mean,5.396537e+07,26.898058,265559.0,49.524089,0.032319,66.686618,0.0,0.142233,0.143497,0.143738,0.142486,0.142257,0.143617,0.142173,0.002035,0.001842,0.001842,0.034330,0.016003,0.016858,0.007285,0.002878,0.009103,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1699.545772,0.020025,0.020073,0.020217,0.019483,0.020133,0.019579,0.019989,0.020073,0.020085,0.018881,0.020025,0.020217,0.010958,0.009031,0.008020,0.019892,0.018821,0.019290,0.020217,0.020217,0.020181,0.010452,0.020217,0.019254,0.020193,0.020157,0.020133,0.020025,0.019724,0.018700,0.020217,0.020217,0.020205,0.020133,0.019760,0.020205,0.008537,0.019302,0.020121,0.020217,0.020217,0.020121,0.020217,0.020217,0.020217,0.020193,0.020133,0.001421,0.014076,0.019266,0.020169,0.020217,0.020133,0.020217,869.528616
std,3.680519e+07,15.609687,0.0,91.940902,0.176847,25.069421,0.0,0.349291,0.350581,0.350826,0.349550,0.349315,0.350704,0.349229,0.045065,0.042883,0.042883,0.182076,0.125487,0.128740,0.085041,0.053569,0.094976,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,964.116155,0.140086,0.140251,0.140744,0.138216,0.140457,0.138550,0.139962,0.140251,0.140292,0.136105,0.140086,0.140744,0.104104,0.094602,0.089193,0.139631,0.135892,0.137544,0.140744,0.140744,0.140621,0.101700,0.140744,0.137418,0.140662,0.140539,0.140457,0.140086,0.139050,0.135465,0.140744,0.140744,0.140703,0.140457,0.139175,0.140703,0.092003,0.137586,0.140416,0.140744,0.140744,0.140416,0.140744,0.140744,0.140744,0.140662,0.140457,0.037668,0.117807,0.137460,0.140580,0.140744,0.140457,0.140744,486.726319
min,9.200000e+01,1.000000,265559.0,-1797.000000,0.000000,26.190000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0

as linhas abaixo de fit e split foram repetidas várias vezes até melhorarmos o indicador rmse do modelo. Assim por fim chegando nos valores de hiperparâmetros descritos acima.

In [17]:
model.fit(X_train, y_train, epochs = 500, batch_size = 10000)
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print(rmse)
print(y_train.mean())

Epoch 1/500
7/7 [==============================] - 1s 19ms/step - loss: 9089.4326 - accuracy: 0.0000e+00
Epoch 2/500
7/7 [==============================] - 0s 18ms/step - loss: 9062.5850 - accuracy: 0.0000e+00
Epoch 3/500
7/7 [==============================] - 0s 16ms/step - loss: 9043.9502 - accuracy: 0.0000e+00
Epoch 4/500
7/7 [==============================] - 0s 16ms/step - loss: 9028.7637 - accuracy: 3.0103e-05
Epoch 5/500
7/7 [==============================] - 0s 19ms/step - loss: 9012.7920 - accuracy: 4.5155e-04
Epoch 6/500
7/7 [==============================] - 0s 19ms/step - loss: 8994.2754 - accuracy: 0.0014
Epoch 7/500
7/7 [==============================] - 0s 17ms/step - loss: 8970.1641 - accuracy: 0.0029
Epoch 8/500
7/7 [==============================] - 0s 19ms/step - loss: 8937.1523 - accuracy: 0.0031
Epoch 9/500
7/7 [==============================] - 0s 16ms/step - loss: 8891.4033 - accuracy: 0.0031
Epoch 10/500
7/7 [==============================] - 0s 16ms/step - loss

In [18]:
pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 1.2 MB/s 


In [19]:
from catboost import CatBoostRegressor as cat

testamos também o modelo de boost com hiperparâmetros mais conservadores pois ao aumentar os valores de hiperparâmetros não obtivemos melhora e no final veremos que o modelo de rede neural performou melhor.

In [20]:
modelboost = cat(depth = 7, iterations=10000)
modelboost.fit(X_train, y_train)
y_predb = model.predict(X_test)
mse = mean_squared_error(y_test, y_predb)
rmse = np.sqrt(mse)
print(rmse)
print(y_train.mean())

A saída de streaming foi truncada nas últimas 5000 linhas.
5003:	learn: 25.6771345	total: 1m 13s	remaining: 1m 13s
5004:	learn: 25.6755072	total: 1m 13s	remaining: 1m 13s
5005:	learn: 25.6747720	total: 1m 13s	remaining: 1m 13s
5006:	learn: 25.6741698	total: 1m 13s	remaining: 1m 13s
5007:	learn: 25.6735637	total: 1m 13s	remaining: 1m 13s
5008:	learn: 25.6695712	total: 1m 13s	remaining: 1m 13s
5009:	learn: 25.6653401	total: 1m 13s	remaining: 1m 13s
5010:	learn: 25.6624287	total: 1m 13s	remaining: 1m 13s
5011:	learn: 25.6601849	total: 1m 13s	remaining: 1m 13s
5012:	learn: 25.6597178	total: 1m 13s	remaining: 1m 13s
5013:	learn: 25.6591712	total: 1m 13s	remaining: 1m 13s
5014:	learn: 25.6587619	total: 1m 13s	remaining: 1m 13s
5015:	learn: 25.6464435	total: 1m 13s	remaining: 1m 13s
5016:	learn: 25.6460162	total: 1m 13s	remaining: 1m 13s
5017:	learn: 25.6456474	total: 1m 13s	remaining: 1m 13s
5018:	learn: 25.6450908	total: 1m 13s	remaining: 1m 13s
5019:	learn: 25.6443213	total: 1m 13s	remaini

In [24]:
item_list = list(traindf['item_nbr'].unique())
#criamos uma lista com os valores de id de item para escalar nosso modelo através de um laço for.
selected_features = ['unit_sales', 'onpromotion',
       'oil_price', 'perishable', 'Friday', 'Monday', 'Saturday', 'Sunday',
       'Thursday', 'Tuesday', 'Wednesday', 'is_additional', 'is_black_friday',
       'is_cyber_monday', 'is_holiday', 'is_holiday_eve',
       'is_natural_disaster', 'is_world_cup', 'Madre', 'Navidad', 'BEVERAGES',
       'BREAD/BAKERY', 'CLEANING', 'GROCERY I', 'PERSONAL CARE', '1004',
       '1010', '1012', '1014', '1040', '1058', '1072', '1084', '1120', '1124',
       '2702', '3032', '3038', '4114', 'transactions', '1', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23',
       '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34',
       '35', '36', '37', '38', '39', '4', '40', '41', '42', '43', '44', '45',
       '46', '47', '48', '49', '5', '50', '51', '52', '53', '54', '6', '7',
       '8', '9','dif_days']

#selecionamos as varáveis       
model = Sequential()
model.add(Dense(30, input_dim=len(selected_features)-1, activation="relu"))
model.add(Dense(20, activation="relu"))
model.add(Dense(10, activation="relu"))
model.add(Dense(1))
model.compile(
    loss='mean_squared_error', 
    optimizer='adam', 
    metrics=['accuracy'])
# definimos as camadas de nosso modelo
selected_features_for_fit = ['unit_sales', 'onpromotion',
       'oil_price', 'perishable', 'Friday', 'Monday', 'Saturday', 'Sunday',
       'Thursday', 'Tuesday', 'Wednesday', 'is_additional', 'is_black_friday',
       'is_cyber_monday', 'is_holiday', 'is_holiday_eve',
       'is_natural_disaster', 'is_world_cup', 'Madre', 'Navidad', 'BEVERAGES',
       'BREAD/BAKERY', 'CLEANING', 'GROCERY I', 'PERSONAL CARE', '1004',
       '1010', '1012', '1014', '1040', '1058', '1072', '1084', '1120', '1124',
       '2702', '3032', '3038', '4114', 'transactions', '1', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23',
       '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34',
       '35', '36', '37', '38', '39', '4', '40', '41', '42', '43', '44', '45',
       '46', '47', '48', '49', '5', '50', '51', '52', '53', '54', '6', '7',
       '8', '9','dif_days']
# nossa variáveis para o dataset de teste       
selected_features_for_test = ['onpromotion',
       'oil_price', 'perishable', 'Friday', 'Monday', 'Saturday', 'Sunday',
       'Thursday', 'Tuesday', 'Wednesday', 'is_additional', 'is_black_friday',
       'is_cyber_monday', 'is_holiday', 'is_holiday_eve',
       'is_natural_disaster', 'is_world_cup', 'Madre', 'Navidad', 'BEVERAGES',
       'BREAD/BAKERY', 'CLEANING', 'GROCERY I', 'PERSONAL CARE', '1004',
       '1010', '1012', '1014', '1040', '1058', '1072', '1084', '1120', '1124',
       '2702', '3032', '3038', '4114', 'transactions', '1', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23',
       '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34',
       '35', '36', '37', '38', '39', '4', '40', '41', '42', '43', '44', '45',
       '46', '47', '48', '49', '5', '50', '51', '52', '53', '54', '6', '7',
       '8', '9','dif_days']
# como vamos rodar item a item precisamos armazenar os resultados do modelo em um dicionário para posteriomente carregar na tabela por id           
dict_item = {}          
listofresults = []
dict_results = {}
#neste laço for selecionamos um id de produto por vez para rodar em nosso modelo
for i in item_list:
  #assim selecionamos o item do dataset principal
  datatestmodel = traindf_dummies[traindf_dummies['item_nbr']==i]
  #selecionamos as variáveis
  features = datatestmodel[selected_features]
  #definimos o index como data
  features.index = datatestmodel['date']
  num_scaler = StandardScaler()
  features_scaled = features
  features_scaled['oil_price'] = num_scaler.fit_transform(features_scaled[['oil_price']])
  features_scaled['transactions'] = num_scaler.fit_transform(features_scaled[['transactions']])
  features_scaled['dif_days'] = num_scaler.fit_transform(features_scaled[['dif_days']])
  #normalizamos as variáveis numpericas
  single_item_data = features_scaled
  #seleciomanos o dataset para fitar nosso modelo
  single_item_test = testdf_dummies[testdf_dummies['item_nbr']==i]
  single_item_fit = single_item_data[selected_features_for_fit]
  #dividimos nosso modelos
  X_train, X_test , y_train, y_test = time_series_train_test_split(single_item_fit,0.2,'unit_sales')
  #fitamos nosso modelo
  model.fit(X_train, y_train, epochs = 500, batch_size = 10000)
  #realizamos a previsão
  y_pred = model.predict(X_test)
  mse = mean_squared_error(y_test, y_pred)
  rmse = np.sqrt(mse)
  result = 'RMSE para o item: '+  str(i) + ' = ' + str(rmse)
  #crimaos a avaliação do modelo
  print(result)
  listofresults.append(result)
  #adicionamos a avaliação em um lista para log
  X_test_f = single_item_test[selected_features_for_test]
  single_item_test['pred_item'] = model.predict(X_test_f)
  dictof = dict (zip(single_item_test['id'], single_item_test['pred_item']))
  dict_result = {i:rmse}
  dict_results ={**dict_results,**dict_result}
  dict_item = {**dict_item , **dictof}
  #realizamos a previsão no df teste final e adicionamos a previsão a um dict para depois carregar em nosso df de teste final.
print(listofresults)

Epoch 1/500


<ipython-input-24-f8e254558297>:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_scaled['oil_price'] = num_scaler.fit_transform(features_scaled[['oil_price']])
<ipython-input-24-f8e254558297>:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_scaled['transactions'] = num_scaler.fit_transform(features_scaled[['transactions']])
<ipython-input-24-f8e254558297>:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer

7/7 [==============================] - 1s 24ms/step - loss: 460.8278 - accuracy: 0.0019
Epoch 2/500
7/7 [==============================] - 0s 19ms/step - loss: 454.2918 - accuracy: 0.0277
Epoch 3/500
7/7 [==============================] - 0s 16ms/step - loss: 446.5105 - accuracy: 0.0556
Epoch 4/500
7/7 [==============================] - 0s 17ms/step - loss: 437.7995 - accuracy: 0.0587
Epoch 5/500
7/7 [==============================] - 0s 17ms/step - loss: 427.9589 - accuracy: 0.0588
Epoch 6/500
7/7 [==============================] - 0s 16ms/step - loss: 416.5487 - accuracy: 0.0588
Epoch 7/500
7/7 [==============================] - 0s 17ms/step - loss: 403.3456 - accuracy: 0.0588
Epoch 8/500
7/7 [==============================] - 0s 18ms/step - loss: 388.5652 - accuracy: 0.0588
Epoch 9/500
7/7 [==============================] - 0s 16ms/step - loss: 372.8044 - accuracy: 0.0588
Epoch 10/500
7/7 [==============================] - 0s 18ms/step - loss: 357.2054 - accuracy: 0.0588
Epoch 11/50

<ipython-input-24-f8e254558297>:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  single_item_test['pred_item'] = model.predict(X_test_f)
<ipython-input-24-f8e254558297>:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_scaled['oil_price'] = num_scaler.fit_transform(features_scaled[['oil_price']])
<ipython-input-24-f8e254558297>:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in 

Epoch 1/500
7/7 [==============================] - 0s 17ms/step - loss: 1449.2335 - accuracy: 0.0302
Epoch 2/500
7/7 [==============================] - 0s 16ms/step - loss: 1063.2085 - accuracy: 0.0307
Epoch 3/500
7/7 [==============================] - 0s 18ms/step - loss: 827.5603 - accuracy: 0.0307
Epoch 4/500
7/7 [==============================] - 0s 17ms/step - loss: 685.0780 - accuracy: 0.0306
Epoch 5/500
7/7 [==============================] - 0s 17ms/step - loss: 587.3608 - accuracy: 0.0300
Epoch 6/500
7/7 [==============================] - 0s 17ms/step - loss: 530.9568 - accuracy: 0.0293
Epoch 7/500
7/7 [==============================] - 0s 18ms/step - loss: 500.0793 - accuracy: 0.0284
Epoch 8/500
7/7 [==============================] - 0s 17ms/step - loss: 480.4418 - accuracy: 0.0286
Epoch 9/500
7/7 [==============================] - 0s 19ms/step - loss: 470.1078 - accuracy: 0.0286
Epoch 10/500
7/7 [==============================] - 0s 17ms/step - loss: 463.1479 - accuracy: 0.02

...

<ipython-input-24-f8e254558297>:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  single_item_test['pred_item'] = model.predict(X_test_f)
<ipython-input-24-f8e254558297>:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_scaled['oil_price'] = num_scaler.fit_transform(features_scaled[['oil_price']])
<ipython-input-24-f8e254558297>:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in 

Epoch 1/500
7/7 [==============================] - 0s 24ms/step - loss: 32279.1504 - accuracy: 0.0241
Epoch 2/500
7/7 [==============================] - 0s 21ms/step - loss: 32277.1055 - accuracy: 0.0241
Epoch 3/500
7/7 [==============================] - 0s 22ms/step - loss: 32275.4277 - accuracy: 0.0241
Epoch 4/500
7/7 [==============================] - 0s 25ms/step - loss: 32274.0508 - accuracy: 0.0241
Epoch 5/500
7/7 [==============================] - 0s 21ms/step - loss: 32272.6777 - accuracy: 0.0241
Epoch 6/500
7/7 [==============================] - 0s 22ms/step - loss: 32271.4375 - accuracy: 0.0241
Epoch 7/500
7/7 [==============================] - 0s 26ms/step - loss: 32270.2344 - accuracy: 0.0241
Epoch 8/500
7/7 [==============================] - 0s 24ms/step - loss: 32269.1562 - accuracy: 0.0241
Epoch 9/500
7/7 [==============================] - 0s 22ms/step - loss: 32268.1250 - accuracy: 0.0241
Epoch 10/500
7/7 [==============================] - 0s 23ms/step - loss: 32267.056

<ipython-input-24-f8e254558297>:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  single_item_test['pred_item'] = model.predict(X_test_f)
<ipython-input-24-f8e254558297>:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_scaled['oil_price'] = num_scaler.fit_transform(features_scaled[['oil_price']])
<ipython-input-24-f8e254558297>:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in 

Epoch 1/500
7/7 [==============================] - 0s 21ms/step - loss: 652.0443 - accuracy: 0.0227
Epoch 2/500
7/7 [==============================] - 0s 25ms/step - loss: 647.5356 - accuracy: 0.0227
Epoch 3/500
7/7 [==============================] - 0s 23ms/step - loss: 634.6628 - accuracy: 0.0227
Epoch 4/500
7/7 [==============================] - 0s 21ms/step - loss: 616.4022 - accuracy: 0.0227
Epoch 5/500
7/7 [==============================] - 0s 25ms/step - loss: 593.7164 - accuracy: 0.0227
Epoch 6/500
7/7 [==============================] - 0s 25ms/step - loss: 573.2327 - accuracy: 0.0227
Epoch 7/500
7/7 [==============================] - 0s 25ms/step - loss: 550.0148 - accuracy: 0.0227
Epoch 8/500
7/7 [==============================] - 0s 23ms/step - loss: 535.3929 - accuracy: 0.0227
Epoch 9/500
7/7 [==============================] - 0s 25ms/step - loss: 522.6168 - accuracy: 0.0227
Epoch 10/500
7/7 [==============================] - 0s 27ms/step - loss: 507.5293 - accuracy: 0.0227

<ipython-input-24-f8e254558297>:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  single_item_test['pred_item'] = model.predict(X_test_f)


In [25]:
testdf_dummies.columns


Index(['id', 'date', 'store_nbr', 'item_nbr', 'onpromotion', 'oil_price',
       'perishable', 'Friday', 'Monday', 'Saturday', 'Sunday', 'Thursday',
       'Tuesday', 'Wednesday', 'is_additional', 'is_black_friday',
       'is_cyber_monday', 'is_holiday', 'is_holiday_eve',
       'is_natural_disaster', 'is_world_cup', 'Madre', 'Navidad', 'BEVERAGES',
       'BREAD/BAKERY', 'CLEANING', 'GROCERY I', 'PERSONAL CARE', '1004',
       '1010', '1012', '1014', '1040', '1058', '1072', '1084', '1120', '1124',
       '2702', '3032', '3038', '4114', 'transactions', '1', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23',
       '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34',
       '35', '36', '37', '38', '39', '4', '40', '41', '42', '43', '44', '45',
       '46', '47', '48', '49', '5', '50', '51', '52', '53', '54', '6', '7',
       '8', '9', 'dif_days'],
      dtype='object')

imprimimos o log de resultados

In [26]:
listofresults

['RMSE para o item: 258396 = 24.50013551323969',
 'RMSE para o item: 261052 = 37.95107131928144',
 'RMSE para o item: 265559 = 121.94871920335741',
 'RMSE para o item: 273528 = 22.63003553562264',
 'RMSE para o item: 314384 = 11.575711519555048',
 'RMSE para o item: 364606 = 31.23503294353388',
 'RMSE para o item: 414353 = 34.9789146426754',
 'RMSE para o item: 414750 = 16.587747325737016',
 'RMSE para o item: 502331 = 15.466919098402519',
 'RMSE para o item: 559870 = 88.39864448669131',
 'RMSE para o item: 564533 = 16.194996725873573',
 'RMSE para o item: 581078 = 8.225680534203585',
 'RMSE para o item: 759893 = 10.752691077200002',
 'RMSE para o item: 839363 = 10.878255979360741',
 'RMSE para o item: 847863 = 10.137855833290786',
 'RMSE para o item: 850333 = 9.220361640010488',
 'RMSE para o item: 1047679 = 56.572101949907605',
 'RMSE para o item: 1084881 = 8.536437115695405',
 'RMSE para o item: 841842 = 26.28789417723818',
 'RMSE para o item: 1036689 = 14.109045631226119']

imporimimos a média para comparar com nosso erro

In [27]:
traindf_dummies.groupby('item_nbr').mean().unit_sales.head(30)

item_nbr
258396     11.555117
261052     29.397437
265559     49.524089
273528     24.857870
314384     42.472630
364606     53.013228
414353     23.606544
414750     24.396554
502331     44.364780
559870     42.814102
564533     28.277684
581078     16.601689
759893     15.002286
839363     17.112590
841842     23.516545
847863     12.509185
850333      9.134851
1036689    21.350525
1047679    68.302194
1084881     9.828267
Name: unit_sales, dtype: float64

In [28]:
testdf['unit_sales'] = testdf['id'].map(lambda x: dict_item[x])

adicionamos a coluna de previsões em nosso df teste

verificando o range de produtos para unidade vendidas

In [29]:
roof = traindf.groupby('item_nbr').max().unit_sales
basement = traindf.groupby('item_nbr').min().unit_sales
rangeofsales = abs(roof) + abs(basement)
rangeofsales.head(30)

item_nbr
258396      2739
261052      2426
265559     14025
273528      2933
314384       612
364606      4636
414353      3158
414750      1851
502331       834
559870     20876
564533      2418
581078       654
759893      4907
839363      9520
841842     44272
847863      3367
850333       686
1036689     1146
1047679     6385
1084881     2400
Name: unit_sales, dtype: int64

criando um df de análise com diversar métricas para verificar o desempenho do erro em relação a outras métricas estatísticas

In [30]:
df_calc_percent_error = traindf_dummies.groupby('item_nbr').mean()
df_calc_percent_error['error'] = df_calc_percent_error.index.map(lambda x: dict_results[x] )
df_calc_percent_error['percent_error'] = df_calc_percent_error['error']/df_calc_percent_error['unit_sales']
df_calc_percent_error['range'] = rangeofsales
df_calc_percent_error['std']=traindf_dummies.groupby('item_nbr').std().unit_sales
df_calc_percent_error['min']=traindf_dummies.groupby('item_nbr').min().unit_sales
traindf['negative_sale'] = traindf['unit_sales'].map(lambda x: x if x <0 else 0)
df_calc_percent_error['negative'] = traindf.groupby('item_nbr').sum().negative_sale
df_calc_percent_error['var_coef'] = df_calc_percent_error['std']/df_calc_percent_error['unit_sales']*100
list(df_calc_percent_error.columns)

['id',
 'store_nbr',
 'unit_sales',
 'onpromotion',
 'oil_price',
 'perishable',
 'Friday',
 'Monday',
 'Saturday',
 'Sunday',
 'Thursday',
 'Tuesday',
 'Wednesday',
 'is_additional',
 'is_black_friday',
 'is_cyber_monday',
 'is_holiday',
 'is_holiday_eve',
 'is_natural_disaster',
 'is_world_cup',
 'Madre',
 'Navidad',
 'BEVERAGES',
 'BREAD/BAKERY',
 'CLEANING',
 'GROCERY I',
 'PERSONAL CARE',
 '1004',
 '1010',
 '1012',
 '1014',
 '1040',
 '1058',
 '1072',
 '1084',
 '1120',
 '1124',
 '2702',
 '3032',
 '3038',
 '4114',
 'transactions',
 '1',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '2',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '3',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '4',
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49',
 '5',
 '50',
 '51',
 '52',
 '53',
 '54',
 '6',
 '7',
 '8',
 '9',
 'dif_days',
 'error',
 'percent_error',
 'range',
 'std',
 'min',
 'negative',
 'var_coef'

selecionando as variáveis de interrese para análise do erro do modelo

In [31]:
selection_df_error = [
 'unit_sales',
     'error',
 'percent_error',
 'range',
 'std',
 'min',
 'negative','var_coef']
df_calc_percent_error= df_calc_percent_error[selection_df_error]
df_calc_percent_error.head(50)

,unit_sales,error,percent_error,range,std,min,negative,var_coef
item_nbr,,,,,,,,
258396,11.555117,24.500136,2.120284,2739,20.083772,-1,-1,173.808466
261052,29.397437,37.951071,1.290965,2426,38.593445,1,0,131.281667
265559,49.524089,121.948719,2.462412,14025,91.940902,-1797,-1904,185.648851
273528,24.857870,22.630036,0.910377,2933,24.800723,-909,-971,99.770104
314384,42.472630,11.575712,0.272545,612,37.041392,1,0,87.212381
364606,53.013228,31.235033,0.589193,4636,46.869328,1,0,88.410629
414353,23.606544,34.978915,1.481747,3158,33.406406,-29,-37,141.513329
414750,24.396554,16.587747,0.679922,1851,33.590908,-1,-1,137.687100
502331,44.364780,15.466919,0.348631,834,29.715692,1,0,66.980366


ordenando a tabela pelo melhor ao pior desempenho de modelos

In [32]:
df_calc_percent_error.sort_values('percent_error').head(50)

,unit_sales,error,percent_error,range,std,min,negative,var_coef
item_nbr,,,,,,,,
314384,42.472630,11.575712,0.272545,612,37.041392,1,0,87.212381
502331,44.364780,15.466919,0.348631,834,29.715692,1,0,66.980366
581078,16.601689,8.225681,0.495473,654,14.907156,1,0,89.793010
564533,28.277684,16.194997,0.572713,2418,27.543024,-1,-1,97.401978
364606,53.013228,31.235033,0.589193,4636,46.869328,1,0,88.410629
839363,17.112590,10.878256,0.635687,9520,48.204246,-1053,-1060,281.688772
1036689,21.350525,14.109046,0.660829,1146,24.649499,-8,-10,115.451490
414750,24.396554,16.587747,0.679922,1851,33.590908,-1,-1,137.687100
759893,15.002286,10.752691,0.716737,4907,23.480607,1,0,156.513531


como era esperado o modelo performou pior com dados mais heterogêneos, porém esta não é a única explicação, temos concorrentemente a variação, alto índice de devoluções também impactando de forma geral negativamente, porém de forma diferente em diferentes produtos de impactar claramente a simplemente não ter impacto nennhum no erro. olhando os agrupamentos por classe de produto e tipo de produto, não observamos pelo menos com os 20 produtos que temos um padrã claro, os dados temos o valor de unidades vendidas que se dá vendas - devoluções, mas como não temos esta separação, seria um  dado interresante para agregar, pois tratando os dois valores em separado poderiamos observar melhor a infuêcia ou não das devoluções nas vendas. outro ponto seria expandir o modelo para poder mensurar de forma melhor a influência das classes e tipos de produtos na assertividade do modelo.

In [33]:
traindf[traindf['unit_sales']<0].head(50)

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,oil_price,perishable,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,is_additional,is_black_friday,is_cyber_monday,is_holiday,is_holiday_eve,is_natural_disaster,is_world_cup,Madre,Navidad,BEVERAGES,BREAD/BAKERY,CLEANING,GROCERY I,PERSONAL CARE,1004,1010,1012,1014,1040,1058,1072,1084,1120,1124,2702,3032,3038,4114,transactions,negative_sale
150597,7052594,2013-06-16,40,559870,-128,0,97.830000,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1054.0,-128
260644,12503035,2013-10-14,18,1047679,-45,0,102.460000,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1366.0,-45
343655,17128964,2014-01-14,18,839363,-7,0,92.150000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1430.0,-7
401342,20766972,2014-03-18,8,414353,-29,0,100.080000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2672.0,-29
432754,22703200,2014-04-21,50,564533,-1,0,104.350000,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2529.0,-1
473528,24957424,2014-06-06,15,841842,-130,0,103.320000,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1283.0,-130
485863,25645777,2014-06-19,44,559870,-20,0,107.080000,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3724.0,-20
536155,28982141,2014-08-13,9,1084881,-8,0,97.570000,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1881.0,-8
659630,38008088,2014-12-23,9,841842,-48,0,56.780000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3525.0,-48
679026,39342094,2015-01-14,39,850333,-1,0,48.490000,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1247.0,-1


In [34]:
traindf.to_csv('traindf_final.csv',index=False)